# **Politecnico di Milano**
## *Student: Caravano Andrea, Alberto Cantele*

*A.Y.: 2024/2025*

*Last modified: 17/03/2025*

### Description: Internet of Things: Challenge n. 1 - Theoretical Exercise: Solution sketch

In [1]:
import math
import numpy as np
import sys

# Prepare problem data
coordinates = [
    [1, 2],
    [10, 3],
    [4, 8],
    [15, 7],
    [6, 1],
    [9, 12],
    [14, 4],
    [3, 10],
    [7, 7],
    [12, 14],
]  # meters

status_update_period = 10 * 60  # 10 minutes, expressed in seconds
packet_size = 2000  # bits
energy_budget = 5 * 10 ** (-3)  # Joule
energy_circuitry = 50 * 10 ** (-9)  # Joule
k_transmission = 1 * 10 ** (-9)  # J/bit/m^2
sink = [20, 20]  # meters

# Fixed value: energy required to operate the TX/RX circuitry, as it does not depend on distance
ec_per_packet = energy_circuitry * packet_size

energies = []
lifetimes = []
for i in range(0, len(coordinates)):
    # Euclidean distance computation ("diagonally") from the sender to the sink node
    distance = math.sqrt(
        (sink[0] - coordinates[i][0]) ** 2 + (sink[1] - coordinates[i][1]) ** 2
    )
    # Energy for transmission per each packet
    etx_per_packet = k_transmission * distance**2 * packet_size
    # Total energy per packet
    energy_per_packet = ec_per_packet + etx_per_packet
    energies.append(energy_per_packet)
    # Let's now compute how many packets (duty cycles, as one per duty cycle is sent) fit in the energy budget
    lifetime = math.floor(energy_budget / energy_per_packet)
    lifetimes.append(lifetime)
    print(
        "Sensor node n. "
        + str(i + 1)
        + " - energy per packet: "
        + str(round(energy_per_packet * 10**3, 3))
        + " mJ - lifetime: "
        + str(lifetime)
        + " cycles = "
        + str(lifetime * status_update_period / 60)
        + " minutes"
    )

# Let's now compute the index of the sensor node that had the worst lifetime!
index_worst = np.argmax(lifetime)
print(
    "\n\nThe sensor node with the worst lifetime is the sensor n. "
    + str(index_worst + 1)
    + " which has a lifetime of "
    + str(lifetimes[index_worst])
    + " cycles = "
    + str(lifetimes[index_worst] * status_update_period / 60)
    + " minutes and energy per packet of "
    + str(round(energies[index_worst] * 10**3, 3))
    + " mJ"
)

Sensor node n. 1 - energy per packet: 1.47 mJ - lifetime: 3 cycles = 30.0 minutes
Sensor node n. 2 - energy per packet: 0.878 mJ - lifetime: 5 cycles = 50.0 minutes
Sensor node n. 3 - energy per packet: 0.9 mJ - lifetime: 5 cycles = 50.0 minutes
Sensor node n. 4 - energy per packet: 0.488 mJ - lifetime: 10 cycles = 100.0 minutes
Sensor node n. 5 - energy per packet: 1.214 mJ - lifetime: 4 cycles = 40.0 minutes
Sensor node n. 6 - energy per packet: 0.47 mJ - lifetime: 10 cycles = 100.0 minutes
Sensor node n. 7 - energy per packet: 0.684 mJ - lifetime: 7 cycles = 70.0 minutes
Sensor node n. 8 - energy per packet: 0.878 mJ - lifetime: 5 cycles = 50.0 minutes
Sensor node n. 9 - energy per packet: 0.776 mJ - lifetime: 6 cycles = 60.0 minutes
Sensor node n. 10 - energy per packet: 0.3 mJ - lifetime: 16 cycles = 160.0 minutes


The sensor node with the worst lifetime is the sensor n. 1 which has a lifetime of 3 cycles = 30.0 minutes and energy per packet of 1.47 mJ


In [2]:
# Constants
MARGIN = 0  # Added margin, to let sink exploration reach the maximum reasonable limit, even if no better results are expected trespassing the boundaries
SINK_MIN_HORIZONTAL = max(
    # Minimum value of the horizontal component of coordinates
    min(coord[0] for coord in coordinates) - MARGIN,
    0,  # Coordinates start from 0, so if the minimum coordinate - the additive component results to be less than 0, we put a fixed limit at 0, of course
)
SINK_MAX_HORIZONTAL = max(coord[0] for coord in coordinates) + MARGIN
SINK_MIN_VERTICAL = max(min(coord[1] for coord in coordinates) - MARGIN, 0)
SINK_MAX_VERTICAL = max(coord[1] for coord in coordinates) + MARGIN

# Final result
lowest_energy_per_packet = sys.float_info.max
optimized_lifetime = None
best_sink = [None, None]

print(
    "Determining the best sink position in the close form (x: (%d -> %d), y: (%d -> %d))"
    % (SINK_MIN_HORIZONTAL, SINK_MAX_HORIZONTAL, SINK_MIN_VERTICAL, SINK_MAX_VERTICAL)
)
# Loop steps of 0.1 (more than that seems unreasonable in a real-world setting)
for i in np.arange(SINK_MIN_HORIZONTAL, SINK_MAX_HORIZONTAL + 0.1, 0.1):
    for j in np.arange(SINK_MIN_VERTICAL, SINK_MAX_VERTICAL + 0.1, 0.1):
        # Temporary sink position
        sink = [i, j]
        # This will contain the set of distances from each sensor to the new temporary sink node
        distance_set = []
        # Again, iterate over each sensor computing the distance
        for k in range(0, len(coordinates)):
            # Euclidean distance computation ("diagonally") from the sender to the sink node
            distance = math.sqrt(
                (sink[0] - coordinates[k][0]) ** 2 + (sink[1] - coordinates[k][1]) ** 2
            )
            # And add it to the local set
            distance_set.append(distance)

        # Energy for transmission per each packet
        etx_per_packet = k_transmission * max(distance_set) ** 2 * packet_size
        # Total energy per packet
        energy_per_packet = ec_per_packet + etx_per_packet
        # Let's now compute how many packets (duty cycles, as one per duty cycle is sent) fit in the energy budget
        lifetime = math.floor(energy_budget / energy_per_packet)
        # Classical minimum energy computation
        if energy_per_packet < lowest_energy_per_packet:
            # The resulting best energy consumption and lifetime values
            lowest_energy_per_packet = energy_per_packet
            optimized_lifetime = lifetime
            # And therefore the sink position is registered as the temporary best
            best_sink = sink

# The result has been finally determinated
print(
    "The best sink position determined in the close form is (x, y) = (%.1f, %.1f), \nwith the worst-performing energy-wise node having consumption of %.2f mJ, corresponding to a lifetime of %d cycles"
    % (best_sink[0], best_sink[1], lowest_energy_per_packet * 10**3, optimized_lifetime)
)

Determining the best sink position in the close form (x: (1 -> 15), y: (1 -> 14))
The best sink position determined in the close form is (x, y) = (6.9, 7.6), 
with the worst-performing energy-wise node having consumption of 0.23 mJ, corresponding to a lifetime of 21 cycles
